In [ ]:
!pip install torch transformers datasets sentencepiece accelerate evaluate rouge_score


In [ ]:
!wget -O data.json https://raw.githubusercontent.com/22bq1a42d4/Linkchat/main/data.json


In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="data.json")["train"]
dataset = dataset.train_test_split(test_size=0.1, seed=42)
print(dataset["train"][0])


In [ ]:
import json, sentencepiece as spm

texts = []
for ex in dataset["train"]:
    texts.append(ex["question"])
    texts.append(ex["answer"])

with open("all_texts.txt", "w", encoding="utf-8") as f:
    for t in texts:
        f.write(t.replace("\n"," ") + "\n")

spm.SentencePieceTrainer.train(
    input='all_texts.txt',
    model_prefix='spm_tokenizer',
    vocab_size=8000,
    model_type='bpe',
    character_coverage=1.0
)

from transformers import T5TokenizerFast
tokenizer = T5TokenizerFast(tokenizer_file="spm_tokenizer.model")


In [ ]:
from transformers import T5Config, T5ForConditionalGeneration

config = T5Config(
    vocab_size=tokenizer.vocab_size,
    d_model=256,
    d_ff=1024,
    num_layers=4,
    num_decoder_layers=4,
    num_heads=4,
    dropout_rate=0.1,
)
model = T5ForConditionalGeneration(config)


In [ ]:
max_input_length = 64
max_target_length = 64

def preprocess(examples):
    inputs = [q.strip() for q in examples["question"]]
    targets = [a.strip() for a in examples["answer"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    labels = tokenizer(text_target=targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized = dataset.map(preprocess, batched=True, remove_columns=dataset["train"].column_names)


In [ ]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import evaluate, numpy as np

rouge = evaluate.load("rouge")
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    return {k: round(v,4) for k,v in result.items()}

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    logging_steps=100,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    num_train_epochs=5,
    learning_rate=5e-4,
    save_total_limit=2,
    fp16=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()


In [ ]:
from transformers import pipeline

pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)
print(pipe("How can I improve my LinkedIn profile?", max_length=64)[0]['generated_text'])


In [ ]:
model.save_pretrained("qa_model")
tokenizer.save_pretrained("qa_tokenizer")

from google.colab import files
files.download("qa_model/config.json")
files.download("qa_tokenizer/tokenizer.json")
